## The Walks of Nathan Lowell
A look at Nate's path to the back gate.

The journey begins [#tommw](https://www.nathanlowell.com/tommw/)

Archive File Structure

``` python
{'meta': {'ids':[
                '12345',
                '67890',
                ...]
          'last_updated': 'UTC Timestamp',
         }
 'data': [
     {'link':URl string,
      'condition': string,
      ...}
         ]
}
 

```


In [1]:
from io import BytesIO
import json
from PIL import Image
import shutil

import arrow
from bs4 import BeautifulSoup
import requests
import twitterscraper as tws

import photo as pto

[#tommw](https://www.nathanlowell.com/tommw/) is a cross channel social media campaign architeched and executed by science fiction author Nathan Lowell. 

There are 4 planks to Nathan's campain:
1. A blog hosted at https://www.nathanlowell.com/tommw/
2. A photo stream hosted on [FlickR](https://www.flickr.com/photos/nlowell/)
3. Podcast feed via [RSS](http://www.nathanlowell.com/tommw/feed/)
4. Posts to twitter with the #tommw hastag on [@nlowell on twitter](https://twitter.com/nlowell)

In [47]:
# Functions to manage the archive file

def get_archive():

    # Load the archive file
    try:
        with open('archive.json', 'r') as file:
            archive= json.loads(file.read())
        print('Archive loaded.')
        
    except:
        # Create archive data structure.

        archive = {'meta':{'ids':[],
                           'audio':[],
                          'last_updated':'New File',
                          },
                  'data':[],
                  }
        print('No archived file, building new archive.json')
        
    return archive

def save_archive(archive):
    
    with open('archive.json', 'w') as file:
        archive['meta']['last_updated'] = str(arrow.now())
        file.write(json.dumps(archive, indent=4))
        
    return True

def save_tweets(archive):
    
    with open('tweets.json', 'w') as file:
        
        file.write(json.dumps(archive, indent=4))
        
    return True

def backfill_dates(archive):
    '''Helper function to convert timestamps
    to a date. Date is used to match audio files to photos.'''

    for walk in archive['data']:

        timestamp = walk['timestamp']

        date = timestamp.split(' ')[0]

        walk['date'] = date

    return archive

    

In [3]:
# Future podcast.py

import requests
import tinytag


def get_mp3(podcast_link, archive):

    mp3_name = podcast_link.split('/')[-1]

    mp3_file = 'audio/' + mp3_name

    if mp3_name in archive['meta']['audio']:
        # skip files already downloaded
        return mp3_file, archive

    file = requests.get(podcast_link)

    if file.status_code != 200:

        return False

    with open(mp3_file, 'wb') as f:
        f.write(file.content)

    archive['meta']['audio'].append(mp3_name)

    return mp3_file, archive



def get_id3_tag(mp3_file):

    id_info = tinytag.TinyTag.get(mp3_file).as_dict()

    title = id_info.get('title')

    if title:
        date = title.split(' ')[-1]
    else:
        date = 'unknown'

    id_info['date'] = date

    id_info['audio'] = mp3_file
    

    return id_info


def match_photo_date(id_info, archive):
    '''Iterates through the archive to match audio
    to a photo record'''

    for walk in archive['data']:

        if walk['date'] == id_info['date']:

            walk.update(id_info)
            mp3_file = id_info.get('audio')
            
            if mp3_file.find('.mp3') > 1:
                mp3_name = mp3_file.strip('audio/')
                archive['meta']['audio'].append(mp3_name)
                
            else:
                print('Unable to get mp3 info from', id_info)

    return archive


def build_audio_archive(podcast_link, archive):
    # TODO complete this function
    # Parse URL into base, date, file_name
    podcast_link.rsplit('/', maxsplit=2)

    pass


In [4]:
# Future Main

# Path to temp directory
latest = 'latest/'

# Import the archive from disk
archive = get_archive()

# Scrape the blog for links to the other media components
blog_page = requests.get("https://www.nathanlowell.com/tommw/")
soup = BeautifulSoup(blog_page.text, 'html.parser')

# Find a flickr link
flickr = soup.find_all(href=True, attrs={'data-flickr-embed':'true'})

# Find a podcast link
podcast_link = soup.find(class_="powerpress_link_pinw").get('href')

if not flickr:
    print('No flickr links found on blog home page.')
    flicker_link = None
    
else:
    flickr_link = flickr[0]['href']

    
if len(archive['meta']['ids']) < 400:
    album_url = 'https://www.flickr.com/photos/nlowell/albums/72157626736309035'
    archive = pto.build_flickr_archive(flickr_link, archive, album=False)
    archive = pto.build_flickr_archive(album_url, archive, album=True)
    

# Process the photostream
stream_url = pto.get_stream_url(flickr_link)
stream = pto.get_photostream(stream_url)
archive = pto.process_photo_stream_page(stream, archive)

# Check the album page
album_url = 'https://www.flickr.com/photos/nlowell/albums/72157626736309035'
stream = pto.get_photostream(album_url, album=True)
archive = pto.process_photo_stream_page(stream, archive)


# Save the audio file
mp3_file, archive = get_mp3(podcast_link, archive)

# Save the data to archive.json
save_archive(archive)
    

Archive loaded.
no #tommow tag found


True

In [ ]:
save_archive(archive)

In [ ]:
# Quick check for blocks of missing photos

times = []
for photo in archive['data']:
    times.append(photo['date'])

times.sort()
times
    

In [ ]:
mp3_file = 'audio/tommw-2018-12-21.mp3'
latest = 'latest/'

In [ ]:
mp3_file.find('.mp3')

In [ ]:
import scipy.io.wavfile as wv
import pydub
import tinytag


In [ ]:
mp3_audio = pydub.AudioSegment.from_mp3(mp3_file)

In [ ]:
#convert to wav
mp3_audio.export(latest+"unprocessed.wav", format="wav")

#read wav file
rate,audData=wv.read(latest+"unprocessed.wav")

# Calculate track time
track_time = (audData.shape[0] / rate)/60



In [ ]:
for walk in archive['data']:
    print(walk['date'])

In [ ]:
archive['data'][0]

In [ ]:
archive = backfill_dates(archive)

In [ ]:
# Future Twitter.py
import toml
import twython as tw

cred = toml.load('credentials.toml')

mm = {'Jan':'01',
     'Feb':'02',
     'Mar':'03',
     'Apr':'04',
     'May':'05',
     'Jun':'06',
     'Jul':'07',
     'Aug':'08',
     'Sep':'09',
     'Oct':'10',
     'Nov':'11',
     'Dec':'12'}

def get_new_twitter_token(cred):
    tw_creds = cred['api']['twitter']

    twitter = tw.Twython(tw_creds['consumer_key'],
                              tw_creds['consumer_secret'],
                              oauth_version=2)

    ACCESS_TOKEN = twitter.obtain_access_token()
    
    cred['api']['twitter']['access_token'] = ACCESS_TOKEN
    
    with open('credentials.toml', 'w') as f:

        toml.dump(cred, f)

    return ACCESS_TOKEN

def get_token(cred):

    token = cred['api']['twitter'].get('access_token')


    if token == None:
        token = get_new_twitter_token(cred)
        
    return token

def get_twitter(token):
    
    twitter = tw.Twython(tw_creds['consumer_key'],
                         access_token=token)

def format_tweet_time(tweet_time):
    '''Transforms tweet date time
    format "5:11 AM - 16 Jun 2011"
    to timestamp "2011-06-16 05:11:00" '''

    dt_parts = tweet_time.split(' ')
    year = dt_parts[-1]
    
    # Convert text month to digits
    month = mm[dt_parts[-2]]
    
    day = dt_parts[-3]
    raw_time = dt_parts[0].split(':')
    hour = raw_time[0].rjust(2,'0')
    minute = raw_time[-1]
    second = '00'

    # Adjust time for am vs pm
    if dt_parts[1]=='PM':
        hour_num = int(hour) + 12
        hour = str(hour)


    tw_date = '-'.join([year,month,day])
    tw_time = ':'.join([hour,minute,second])
    tw_timestamp = ' '.join([tw_date, tw_time])

    return tw_timestamp, tw_date, tw_time

def clean_twsrapper_time(tweet_data):
    '''Converts date time object in scrapper dict data to string'''
    
    for tweet in tweet_data:
        tweet['timestamp'] = datetime.datetime.strftime(tweet['timestamp'], '%Y-%m-%d %H:%M:%S')
        tweet['date'] = tweet['timestamp'].split(' ')[0]
        
    return tweet_data


def process_tweet_body(body):

    info = {}
    
    raw_title = body.split('"')[1]

    comma = raw_title.find(',')

    # Find a period with a space after it.
    period = raw_title.find('. ')

    if comma > 0:
        tag_temp = raw_title[:comma]
        wind_link = raw_title[comma:]

    elif comma < 0 and period > 0:

        tag_temp = raw_title[:period]
        wind_link = raw_title[period:]

    else:
        print('Unable to parse title')
        info['tag'] = 'Error'
        return info

    info['tw_title'] = raw_title
    
    # TODO refactor using deque to improve performance    
    t_parts = tag_temp.split(' ')
    info['tag'] = t_parts.pop(0)
    info['temp'] = t_parts.pop(0)
    info['sun'] = ' '.join(t_parts)

    w_parts = wind_link.split(' ')
    info['photo_link'] = w_parts.pop()
    info['wind'] = ' '.join(w_parts).strip(', ')
    
    return info

def process_tweet(tweet_url):
    '''Extracts text, timestamp and date from a tweet by url.'''
    
    tweet = requests.get(tweet_url)
    tsoup = BeautifulSoup(tweet.text, 'html.parser')
    
    body = tsoup.title.text
    info = process_tweet_body(body)
    
    tweet_time = str(tsoup.find(class_="client-and-actions").text.strip())
    tw_time = format_tweet_time(tweet_time)
    
    info['tweet_timestamp'], info['date'], info['time'] = tw_time
    
    info['tweet_id'] = tweet_url.split('/')[-1]
    
    return info

if __name__ == "__main__":
    
    token = get_token(cred)
        


In [ ]:
search_url = 'https://twitter.com/search?l=en&q=%23tommw%20from%3Anlowell%20since%3A2011-06-01%20until%3A2011-07-31&src=typd'

tweet_url = 'https://twitter.com/nlowell/status/81333156376096768'
tw_info = process_tweet(tweet_url)
tw_info

In [ ]:
search_results = requests.get(search_url)

In [44]:
import datetime

# Playing with twitter scraper
# tommw_search = tws.query_tweets('#tommw', 20)

tweet_data = []

for y in range(2011,2020):
    
    
    for m in range(1,13):
        
        e = m+1
        ye = y
        
        if e == 13:
            e = 1
            ye = y+1

        tommw_search = tws.query_tweets('#tommw', limit=None,
                         begindate=datetime.date(y, m, 2),
                         enddate=datetime.date(ye, e, 1), poolsize=30, lang='')

        for tweet in tommw_search:

            tweet.timestamp = datetime.datetime.strftime(
                                tweet['timestamp'],
                                '%Y-%m-%d %H:%M:%S')
            
            tweet.date = tweet.timestamp.split(' ')[0] # Not tested jan-2019
            if tweet.user == 'nlowell':
                
                tweet_data.append(vars(tweet))




INFO: queries: ['#tommw since:2011-01-02 until:2011-01-03', '#tommw since:2011-01-03 until:2011-01-04', '#tommw since:2011-01-04 until:2011-01-05', '#tommw since:2011-01-05 until:2011-01-06', '#tommw since:2011-01-06 until:2011-01-07', '#tommw since:2011-01-07 until:2011-01-08', '#tommw since:2011-01-08 until:2011-01-09', '#tommw since:2011-01-09 until:2011-01-10', '#tommw since:2011-01-10 until:2011-01-11', '#tommw since:2011-01-11 until:2011-01-12', '#tommw since:2011-01-12 until:2011-01-13', '#tommw since:2011-01-13 until:2011-01-14', '#tommw since:2011-01-14 until:2011-01-15', '#tommw since:2011-01-15 until:2011-01-16', '#tommw since:2011-01-16 until:2011-01-17', '#tommw since:2011-01-17 until:2011-01-18', '#tommw since:2011-01-18 until:2011-01-19', '#tommw since:2011-01-19 until:2011-01-20', '#tommw since:2011-01-20 until:2011-01-21', '#tommw since:2011-01-21 until:2011-01-22', '#tommw since:2011-01-22 until:2011-01-23', '#tommw since:2011-01-23 until:2011-01-24', '#tommw since:20

INFO: Querying #tommw since:2011-02-12 until:2011-02-13
INFO: Querying #tommw since:2011-02-26 until:2011-02-27
INFO: Querying #tommw since:2011-02-20 until:2011-02-21
INFO: Querying #tommw since:2011-02-24 until:2011-02-25
INFO: Querying #tommw since:2011-02-10 until:2011-02-11
INFO: Querying #tommw since:2011-02-11 until:2011-02-12
INFO: Querying #tommw since:2011-02-05 until:2011-02-06
INFO: Querying #tommw since:2011-02-21 until:2011-02-22
INFO: Querying #tommw since:2011-02-02 until:2011-02-03
INFO: Querying #tommw since:2011-02-03 until:2011-02-04
INFO: Querying #tommw since:2011-02-28 until:2011-03-01
INFO: Got 0 tweets for %23tommw%20since%3A2011-02-12%20until%3A2011-02-13.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-02-15%20until%3A2011-02-16.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-02-26%20until%3A2011-02-27.
INFO: Got 0 tweets for %23tommw%20since%3A2011-02-05%20until%3A2011-02-06.
INFO: Got 0 tweets for %23to

INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-03-05%20until%3A2011-03-06.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-03-11%20until%3A2011-03-12.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-03-10%20until%3A2011-03-11.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-03-04%20until%3A2011-03-05.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2011-03-18%20until%3A2011-03-19.
INFO: Got 0 tweets (0 new).
INFO: Got 3 tweets for %23tommw%20since%3A2011-03-30%20until%3A2011-03-31.
INFO: Got 3 tweets (3 new).
INFO: Got 4 tweets for %23tommw%20since%3A2011-03-25%20until%3A2011-03-26.
INFO: Got 7 tweets (4 new).
INFO: Got 6 tweets for %23tommw%20since%3A2011-03-28%20until%3A2011-03-29.
INFO: Got 13 tweets (6 new).
INFO: Got 6 tweets for %23tommw%20since%3A2011-03-27%20until%3A2011-03-28.
INFO: Got 19 tweets (6 new).
INFO: Got 6 tweets for %23tommw%20since%3A2

INFO: Querying #tommw since:2011-05-21 until:2011-05-22
INFO: Querying #tommw since:2011-05-25 until:2011-05-26
INFO: Querying #tommw since:2011-05-28 until:2011-05-29
INFO: Querying #tommw since:2011-05-16 until:2011-05-17
INFO: Querying #tommw since:2011-05-22 until:2011-05-23
INFO: Querying #tommw since:2011-05-07 until:2011-05-08
INFO: Querying #tommw since:2011-05-09 until:2011-05-10
INFO: Querying #tommw since:2011-05-11 until:2011-05-12
INFO: Querying #tommw since:2011-05-13 until:2011-05-14
INFO: Querying #tommw since:2011-05-08 until:2011-05-09
INFO: Querying #tommw since:2011-05-18 until:2011-05-19
INFO: Querying #tommw since:2011-05-06 until:2011-05-07
INFO: Querying #tommw since:2011-05-23 until:2011-05-24
INFO: Querying #tommw since:2011-05-05 until:2011-05-06
INFO: Querying #tommw since:2011-05-10 until:2011-05-11
INFO: Querying #tommw since:2011-05-04 until:2011-05-05
INFO: Querying #tommw since:2011-05-24 until:2011-05-25
INFO: Querying #tommw since:2011-05-14 until:201

INFO: Got 1 tweets for %23tommw%20since%3A2011-06-12%20until%3A2011-06-13.
INFO: Got 4 tweets (1 new).
INFO: Got 5 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-20%20until%3A2011-06-21.
INFO: Got 6 tweets (1 new).
INFO: Got 7 tweets (1 new).
INFO: Got 8 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-26%20until%3A2011-06-27.
INFO: Got 9 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-29%20until%3A2011-06-30.
INFO: Got 10 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-25%20until%3A2011-06-26.
INFO: Got 11 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-24%20until%3A2011-06-25.
INFO: Got 12 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-21%20until%3A2011-06-22.
INFO: Got 13 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-13%20until%3A2011-06-14.
INFO: Got 14 tweets (1 new).
INFO: Got 1 tweets for %23tommw%20since%3A2011-06-30%20until%3A2011-07-01.
INFO: Got 15

...

INFO: Got 0 tweets for %23tommw%20since%3A2019-06-11%20until%3A2019-06-12.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-08%20until%3A2019-06-09.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-15%20until%3A2019-06-16.
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-07%20until%3A2019-06-08.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-12%20until%3A2019-06-13.
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-29%20until%3A2019-06-30.
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-18%20until%3A2019-06-19.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-21%20until%3A2019-06-22.
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-06%20until%3A2019-06-07.
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-17%20until%3A2019-06-18.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-06-27%20until%3A2019-06-28.
INFO: Got 0 tweets for %23tommw%20s

INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets (0 new).
INFO: queries: ['#tommw since:2019-08-02 until:2019-08-03', '#tommw since:2019-08-03 until:2019-08-04', '#tommw since:2019-08-04 until:2019-08-05', '#tommw since:2019-08-05 until:2019-08-06', '#tommw since:2019-08-06 until:2019-08-07', '#tommw since:2019-08-07 until:2019-08-08', '#tommw since:2019-08-08 until:2019-08-09', '#tommw since:2019-08-09 until:2019-08-10', '#tommw since:2019-08-10 until:2019-08-11', '#tommw since:2019-08-11 until:2019-08-12', '#tommw since:2019-08-12 until:2019-08-13', '#tommw since:2019-08-13 until:2019-08-14', '#tommw since:2019-08-14 until:2019-08-15', '#tommw since:2019-08-15 until:2019-08-16', '#tommw since:2019-08-16 until:2

INFO: Querying #tommw since:2019-09-04 until:2019-09-05
INFO: Querying #tommw since:2019-09-17 until:2019-09-18
INFO: Querying #tommw since:2019-09-29 until:2019-09-30
INFO: Querying #tommw since:2019-09-30 until:2019-10-01
INFO: Querying #tommw since:2019-09-07 until:2019-09-08
INFO: Querying #tommw since:2019-09-08 until:2019-09-09
INFO: Querying #tommw since:2019-09-13 until:2019-09-14
INFO: Querying #tommw since:2019-09-24 until:2019-09-25
INFO: Querying #tommw since:2019-09-10 until:2019-09-11
INFO: Querying #tommw since:2019-09-21 until:2019-09-22
INFO: Querying #tommw since:2019-09-18 until:2019-09-19
INFO: Querying #tommw since:2019-09-06 until:2019-09-07
INFO: Querying #tommw since:2019-09-03 until:2019-09-04
INFO: Querying #tommw since:2019-09-11 until:2019-09-12
INFO: Querying #tommw since:2019-09-05 until:2019-09-06
INFO: Querying #tommw since:2019-09-28 until:2019-09-29
INFO: Querying #tommw since:2019-09-12 until:2019-09-13
INFO: Querying #tommw since:2019-09-02 until:201

INFO: Got 0 tweets for %23tommw%20since%3A2019-10-22%20until%3A2019-10-23.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-04%20until%3A2019-10-05.
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-12%20until%3A2019-10-13.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-18%20until%3A2019-10-19.
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-09%20until%3A2019-10-10.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-30%20until%3A2019-10-31.
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-08%20until%3A2019-10-09.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-05%20until%3A2019-10-06.
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-24%20until%3A2019-10-25.
INFO: Got 0 tweets (0 new).
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-19%20until%3A2019-10-20.
INFO: Got 0 tweets for %23tommw%20since%3A2019-10-07%20until%3A2019-10-08.
INFO: Got 0 tweets (0 new).
INFO: G

INFO: Querying #tommw since:2019-12-28 until:2019-12-29
INFO: Querying #tommw since:2019-12-29 until:2019-12-30
INFO: Querying #tommw since:2019-12-25 until:2019-12-26
INFO: Querying #tommw since:2019-12-16 until:2019-12-17
INFO: Querying #tommw since:2019-12-07 until:2019-12-08
INFO: Querying #tommw since:2019-12-22 until:2019-12-23
INFO: Querying #tommw since:2019-12-23 until:2019-12-24
INFO: Querying #tommw since:2019-12-17 until:2019-12-18
INFO: Querying #tommw since:2019-12-08 until:2019-12-09
INFO: Querying #tommw since:2019-12-10 until:2019-12-11
INFO: Querying #tommw since:2019-12-13 until:2019-12-14
INFO: Querying #tommw since:2019-12-30 until:2019-12-31
INFO: Querying #tommw since:2019-12-21 until:2019-12-22
INFO: Querying #tommw since:2019-12-27 until:2019-12-28
INFO: Querying #tommw since:2019-12-12 until:2019-12-13
INFO: Querying #tommw since:2019-12-03 until:2019-12-04
INFO: Querying #tommw since:2019-12-20 until:2019-12-21
INFO: Querying #tommw since:2019-12-26 until:201

In [30]:
tweet_data = []
for tweet in tommw_search:
    
    tweet.timestamp = tweet.timestamp
    tweet_data.append(vars(tweet))

In [43]:
m

12

In [ ]:
archive = get_archive()
archive['data'][20]

In [35]:
for y in range(2011,2020):
    print(y)
    
    for m in range(1,13):
        print(m)
    

2011
1
2
3
4
5
6
7
8
9
10
11
12
2012
1
2
3
4
5
6
7
8
9
10
11
12
2013
1
2
3
4
5
6
7
8
9
10
11
12
2014
1
2
3
4
5
6
7
8
9
10
11
12
2015
1
2
3
4
5
6
7
8
9
10
11
12
2016
1
2
3
4
5
6
7
8
9
10
11
12
2017
1
2
3
4
5
6
7
8
9
10
11
12
2018
1
2
3
4
5
6
7
8
9
10
11
12
2019
1
2
3
4
5
6
7
8
9
10
11
12


In [59]:

tweet

{'user': 'nlowell',
 'fullname': 'Nathan Lowell',
 'id': '1083385343720804352',
 'url': '/nlowell/status/1083385343720804352',
 'timestamp': '2019-01-10 15:29:28',
 'text': '#tommw 32F scattered clouds. Calm pic.twitter.com/LBSggZyngp',
 'replies': 1,
 'retweets': 0,
 'likes': 8,
 'html': '<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="en"><a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/tommw?src=hash"><s>#</s><b><strong>tommw</strong></b></a> 32F scattered clouds. Calm <a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/LBSggZyngp">pic.twitter.com/LBSggZyngp</a></p>',
 'date': '2019-01-10'}

In [60]:
save_tweets(tweet_data)

True